In [ ]:
import numpy as np
from scipy.fft import rfft, irfft, rfftfreq
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from ipywidgets import (
    FloatSlider, Checkbox, IntText, FloatText, Layout, HBox, VBox, Output
)
from IPython.display import display, clear_output

# Define the power law PSD contributions and Gaussian peak
def generate_power_law_psd(frequencies, exponent, amplitude=1.0, randomness=0.1):
    psd = np.zeros_like(frequencies)
    non_zero_frequencies = frequencies > 0
    base_psd = amplitude * (frequencies[non_zero_frequencies] ** exponent)
    random_factor = 1 + randomness * np.random.randn(len(base_psd))
    psd[non_zero_frequencies] = base_psd * random_factor
    return psd

def add_gaussian_peak(frequencies, peak_location, peak_amplitude, peak_width):
    # Hard-coded peak_randomness to 0.2
    peak_randomness = 0.2
    gaussian_shape = peak_amplitude * np.exp(-((frequencies - peak_location) ** 2) / (2 * peak_width ** 2))
    random_factor = 1 + peak_randomness * np.random.randn(len(frequencies))
    psd_peak = gaussian_shape * random_factor
    return psd_peak

# Corrected autocorrelation PSD function with proper normalization
def compute_autocorrelation_psd(electric_field, dt):
    N = len(electric_field)
    R_E = np.correlate(electric_field, electric_field, mode='full')[N-1:]
    autocorr_laser_psd = np.abs(rfft(R_E)) * dt / N
    return autocorr_laser_psd

# Compute the PSD and autocorrelation-based PSD
def compute_psd(
    N, T, f_central, include_f3, amp_f3, rand_f3,
    include_f2, amp_f2, rand_f2, 
    include_f1, amp_f1, rand_f1, 
    include_f0, amp_f0, rand_f0,
    include_fp1, amp_fp1, rand_fp1,
    include_fp2, amp_fp2, rand_fp2,
    include_peak, peak_location, peak_amplitude, peak_width
):
    dt = T / N
    frequencies = rfftfreq(N, dt)

    # Ensure f_central and peak_location are within allowable range
    f_min = 1 / T
    f_max = N / (2 * T)
    f_central = np.clip(f_central, f_min, f_max)
    peak_location = np.clip(peak_location, f_min, f_max)
    
    psd_contributions = []

    # Add selected PSD contributions
    if include_f3:
        psd_contributions.append(generate_power_law_psd(frequencies, exponent=-3, amplitude=amp_f3, randomness=rand_f3))
    if include_f2:
        psd_contributions.append(generate_power_law_psd(frequencies, exponent=-2, amplitude=amp_f2, randomness=rand_f2))
    if include_f1:
        psd_contributions.append(generate_power_law_psd(frequencies, exponent=-1, amplitude=amp_f1, randomness=rand_f1))
    if include_f0:
        psd_contributions.append(generate_power_law_psd(frequencies, exponent=0, amplitude=amp_f0, randomness=rand_f0))
    if include_fp1:
        psd_contributions.append(generate_power_law_psd(frequencies, exponent=1, amplitude=amp_fp1, randomness=rand_fp1))
    if include_fp2:
        psd_contributions.append(generate_power_law_psd(frequencies, exponent=2, amplitude=amp_fp2, randomness=rand_fp2))

    if psd_contributions:
        total_psd = np.sum(psd_contributions, axis=0)
    else:
        total_psd = np.ones_like(frequencies) * 1e-12  # Minimal PSD if no contributions are selected

    if include_peak:
        # peak_randomness is now hard-coded within add_gaussian_peak
        peak_psd = add_gaussian_peak(frequencies, peak_location, peak_amplitude, peak_width)
        total_psd += peak_psd
    
    # Generate noise in frequency domain and transform to time domain
    noise_freq_domain = (
        np.random.normal(size=len(frequencies)) + 
        1j * np.random.normal(size=len(frequencies))
    ) * np.sqrt(total_psd)
    noise_time_domain = irfft(noise_freq_domain, N)
    
    # Generate electric field with noise
    time = np.arange(N) * dt
    electric_field = np.cos(2 * np.pi * f_central * time + noise_time_domain)
    
    # Compute FFT-based PSD
    electric_field_fft = np.abs(rfft(electric_field))**2
    laser_psd = electric_field_fft / N
    
    # Compute autocorrelation-based PSD
    autocorr_laser_psd = compute_autocorrelation_psd(electric_field, dt)

    # Plot the results
    plot_all_subplots(frequencies, total_psd, time, electric_field, laser_psd, autocorr_laser_psd, f_central)

# Plotting functions with GridSpec for layout flexibility
def plot_all_subplots(frequencies, total_psd, time, electric_field, laser_psd, autocorr_laser_psd, f_central):
    fig = plt.figure(figsize=(14, 12), constrained_layout=True)
    gs = GridSpec(3, 2, figure=fig, height_ratios=[1, 1, 1])

    # PSD plot
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.loglog(frequencies[frequencies > 0], total_psd[frequencies > 0])
    ax1.set_title('Simulated Frequency Noise PSD')
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('PSD')
    ax1.grid(True)

    # Electric field plot
    ax2 = fig.add_subplot(gs[0, 1])
    num_periods = 30
    max_time = num_periods / f_central
    time_points = time[time <= max_time]
    electric_field_points = electric_field[:len(time_points)]
    if len(electric_field_points) > 1:
        ax2.plot(time_points, electric_field_points)
    else:
        ax2.text(
            0.5, 0.5, 
            'Insufficient data for 30 periods\nTry adjusting parameters', 
            ha='center', va='center'
        )
    ax2.set_title('Time-Dependent Electric Field (Noisy Laser)')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Electric Field Amplitude')
    ax2.grid(True)

    # Laser PSD plot
    ax3 = fig.add_subplot(gs[1, 0])
    ax3.loglog(frequencies[frequencies > 0], laser_psd[frequencies > 0])
    ax3.set_title('PSD of the Noisy Laser')
    ax3.set_xlabel('Frequency (Hz)')
    ax3.set_ylabel('Power Spectral Density')
    ax3.grid(True)

    # Autocorrelation-based PSD plot
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.loglog(frequencies[frequencies > 0], autocorr_laser_psd[frequencies > 0])
    ax4.set_title('PSD from Autocorrelation Function')
    ax4.set_xlabel('Frequency (Hz)')
    ax4.set_ylabel('Power Spectral Density')
    ax4.grid(True)

    plt.show()

# Main function to arrange widgets and handle updates
def Examine_Frequency_Noise():
    # Create output widget
    output = Output()

    # Define layout for widgets
    slider_layout = Layout(width='300px')

    # Define primary widgets
    N_widget = IntText(
        value=4096, 
        description="Num Time Points:", 
        layout=slider_layout
    )
    T_widget = FloatText(
        value=1e-3, 
        description="Total Time (s):", 
        layout=slider_layout
    )
    
    # Initial frequency calculations
    f_min_initial = 1 / T_widget.value
    f_max_initial = N_widget.value / (2 * T_widget.value)
    
    # Define frequency sliders
    f_central_slider = FloatSlider(
        min=f_min_initial, 
        max=f_max_initial, 
        step=f_min_initial, 
        value=f_min_initial, 
        description="Central Freq (Hz):", 
        continuous_update=False,  # Changed to False for performance
        layout=slider_layout
    )
    
    # Rename peak_location_slider to noise_peak_central_freq_slider for clarity
    noise_peak_central_freq_slider = FloatSlider(
        min=f_min_initial, 
        max=f_max_initial, 
        step=f_min_initial, 
        value=f_min_initial, 
        description="Noise Peak Central Freq (Hz):",  # Updated description
        continuous_update=False,  # Changed to False for performance
        layout=slider_layout
    )
    
    # Define PSD contribution widgets for exponents -3 to +2
    def create_psd_widgets(exponent):
        include = Checkbox(
            value=False, 
            description=f"Include $f^{{{exponent}}}$"
        )
        amp = FloatText(
            min=1e-8, 
            max=10, 
            step=1e-4, 
            value=1e-4 if exponent >=0 else 1e-2, 
            description=f"Amp $f^{{{exponent}}}$", 
            layout=slider_layout
        )
        rand = FloatSlider(
            min=0, 
            max=0.2, 
            step=0.01, 
            value=0.1, 
            description=f"Rand $f^{{{exponent}}}$", 
            continuous_update=False,  # Changed to False for performance
            layout=slider_layout
        )
        return include, amp, rand

    # Create widgets for each exponent
    exponents = [-3, -2, -1, 0, 1, 2]
    psd_widgets = {}
    for exp in exponents:
        include, amp, rand = create_psd_widgets(exp)
        psd_widgets[exp] = (include, amp, rand)
    
    # Define peak widgets
    include_peak_checkbox = Checkbox(
        value=False, 
        description="Include Noise Peak"
    )
    peak_amplitude_slider = FloatSlider(
        min=1e-8, 
        max=1e4, 
        step=1e-4, 
        value=1, 
        description="Peak Amplitude", 
        continuous_update=False,  # Changed to False for performance
        layout=slider_layout
    )
    # Removed peak_randomness_slider as per requirement
    peak_width_slider = FloatSlider(
        min=1e3, 
        max=1e6, 
        step=1e3, 
        value=1e4, 
        description="Peak Width (Hz)", 
        continuous_update=False,  # Changed to False for performance
        layout=slider_layout
    )
    
    # Function to update frequency sliders based on N and T
    def update_frequency_sliders(*args):
        new_f_min = 1 / T_widget.value
        new_f_max = N_widget.value / (2 * T_widget.value)
        
        # Update f_central_slider
        f_central_slider.min = new_f_min
        f_central_slider.max = new_f_max
        f_central_slider.step = new_f_min
        f_central_slider.value = np.clip(f_central_slider.value, new_f_min, new_f_max)
        
        # Update noise_peak_central_freq_slider
        noise_peak_central_freq_slider.min = new_f_min
        noise_peak_central_freq_slider.max = new_f_max
        noise_peak_central_freq_slider.step = new_f_min
        noise_peak_central_freq_slider.value = np.clip(noise_peak_central_freq_slider.value, new_f_min, new_f_max)
    
    # Function to compute and display PSD
    def compute_on_update(*args):
        with output:
            clear_output(wait=True)
            compute_psd(
                N=N_widget.value,
                T=T_widget.value,
                f_central=f_central_slider.value,
                include_f3=psd_widgets[-3][0].value,
                amp_f3=psd_widgets[-3][1].value,
                rand_f3=psd_widgets[-3][2].value,
                include_f2=psd_widgets[-2][0].value,
                amp_f2=psd_widgets[-2][1].value,
                rand_f2=psd_widgets[-2][2].value,
                include_f1=psd_widgets[-1][0].value,
                amp_f1=psd_widgets[-1][1].value,
                rand_f1=psd_widgets[-1][2].value,
                include_f0=psd_widgets[0][0].value,
                amp_f0=psd_widgets[0][1].value,
                rand_f0=psd_widgets[0][2].value,
                include_fp1=psd_widgets[1][0].value,
                amp_fp1=psd_widgets[1][1].value,
                rand_fp1=psd_widgets[1][2].value,
                include_fp2=psd_widgets[2][0].value,
                amp_fp2=psd_widgets[2][1].value,
                rand_fp2=psd_widgets[2][2].value,
                include_peak=include_peak_checkbox.value,
                peak_location=noise_peak_central_freq_slider.value,  # Updated parameter
                peak_amplitude=peak_amplitude_slider.value,
                peak_width=peak_width_slider.value
            )

    # Attach observers to N and T to update frequency sliders
    N_widget.observe(update_frequency_sliders, names='value')
    T_widget.observe(update_frequency_sliders, names='value')
    
    # Attach observers to all widgets to trigger compute_on_update
    # Primary widgets
    widgets_to_observe = [
        N_widget, T_widget, f_central_slider, noise_peak_central_freq_slider,
        include_peak_checkbox, peak_amplitude_slider, peak_width_slider
    ]
    
    # PSD contribution widgets
    for exp in exponents:
        include, amp, rand = psd_widgets[exp]
        widgets_to_observe.extend([include, amp, rand])
    
    for widget in widgets_to_observe:
        widget.observe(compute_on_update, names='value')
    
    # Arrange PSD contribution widgets
    psd_contrib_boxes = []
    for exp in exponents:
        include, amp, rand = psd_widgets[exp]
        box = HBox([include, amp, rand])
        psd_contrib_boxes.append(box)
    
    # Arrange all controls in a VBox with Noise Peak Central Frequency slider at the bottom
    controls_box = VBox([
        HBox([N_widget, T_widget]),
        HBox([f_central_slider]),  # Only Central Freq Slider here
        VBox(psd_contrib_boxes),
        HBox([noise_peak_central_freq_slider]),  # Moved Noise Peak Central Freq Slider here
        VBox([
            include_peak_checkbox,
            HBox([peak_amplitude_slider, peak_width_slider])  # Removed peak_randomness_slider
        ])
    ])
    
    # Display the widgets and output area
    display(controls_box)
    display(output)
    
    # Perform initial computation
    compute_on_update()

# Initialize the interactive frequency noise analyzer
Examine_Frequency_Noise()
